# Import Packages

In [1]:
# pip install sdv

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import re
import time
from sdv.tabular import GaussianCopula, CTGAN, CopulaGAN, TVAE
from sdv.constraints import Inequality, create_custom_constraint
from copy import deepcopy
import warnings
from itertools import combinations
from sd_generator import SD_generator

In [3]:
warnings.filterwarnings("ignore")

# Prepare Data

In [4]:
data = pd.read_csv('data.csv')
data.drop(columns=['Name'], axis=1, inplace=True)
data.head()

,Date_of_birth,Opening_date,Expiry_date,Credit_limit,Statement_balance,Available_credit
0,1949-08-17,2008-09-02,2013-09-02,38000.0,29297.69,8702.31
1,1938-07-04,2019-09-08,2024-09-08,19500.0,15628.50,3871.50
2,1927-03-28,2019-04-23,2024-04-23,11500.0,7388.34,4111.66
3,1936-01-17,2021-10-27,2026-10-27,68500.0,32060.39,36439.61
4,1960-09-05,2010-10-02,2015-10-02,72000.0,56148.13,15851.87


In [5]:
data['Opening'] = data['Opening_date'].apply(lambda x: x.replace('-', ''))
data['Expiry'] = data['Expiry_date'].apply(lambda x: x.replace('-', ''))
data['Client_id'] = data['Opening'] + data['Expiry']
data['Client_id'] = data['Client_id'].apply(lambda x: int(x))
data['Opening'] = data['Opening'].apply(lambda x: int(x))
data['Expiry'] = data['Expiry'].apply(lambda x: int(x))

data = data[['Client_id', 'Date_of_birth', 'Opening_date', 'Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit', 'Opening', 'Expiry']]

data.head()

,Client_id,Date_of_birth,Opening_date,Expiry_date,Credit_limit,Statement_balance,Available_credit,Opening,Expiry
0,2008090220130902,1949-08-17,2008-09-02,2013-09-02,38000.0,29297.69,8702.31,20080902,20130902
1,2019090820240908,1938-07-04,2019-09-08,2024-09-08,19500.0,15628.50,3871.50,20190908,20240908
2,2019042320240423,1927-03-28,2019-04-23,2024-04-23,11500.0,7388.34,4111.66,20190423,20240423
3,2021102720261027,1936-01-17,2021-10-27,2026-10-27,68500.0,32060.39,36439.61,20211027,20261027
4,2010100220151002,1960-09-05,2010-10-02,2015-10-02,72000.0,56148.13,15851.87,20101002,20151002


In [6]:
data.dtypes

Client_id              int64
Date_of_birth         object
Opening_date          object
Expiry_date           object
Credit_limit         float64
Statement_balance    float64
Available_credit     float64
Opening                int64
Expiry                 int64
dtype: object

# Compare Synthetic Data Generated with & without Constraints Applied

## Generate Synthetic Data with Constraints

In [7]:
d1 = SD_generator(data, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d1.preprocess()
d1.detect_inequality()
d1.detect_arithmetic_equality()
d1.detect_arithmetic_inequality()
d1.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0945 seconds
8 relationships detected
Execution Time: 0.5657 seconds
1 relationships detected
Execution Time: 1.3636 seconds
8 relationships detected
Execution Time: 0.5952 seconds
2 relationships detected
Execution Time: 0.4549 seconds


### Original Relationships Detected:

In [8]:
print("Inequality: ", d1.inequality_dict)
print("Arithmetic Equality: ", d1.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d1.arithmetic_inequality_dict)
print("Inclusive: ", d1.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Opening': ['Expiry_date', 'Credit_limit'], 'Expiry': ['Opening'], 'Credit_limit': ['Statement_balance', 'Available_credit']}
Arithmetic Equality:  {'Credit_limit': [['Statement_balance', 'Available_credit']]}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2000010600000000.0]], 'Opening_date': [['Date_of_birth', 6599.0]], 'Expiry_date': [['Opening_date', 1826.0]], 'Opening': [['Expiry_date', 19961750.0], ['Credit_limit', 19900301.0]], 'Expiry': [['Opening', 49999.0]], 'Credit_limit': [['Statement_balance', 3.9400000000023283], ['Available_credit', 7.260000000000218]]}
Inclusive:  {'Client_id': [['Opening', 0], ['Expiry', 8]]}


### Create Models

In [9]:
d1.create_constraints(inequality=True, arithmetic_equality=True, arithmetic_inequality=True, inclusive=True)
d1.apply_model("GaussianCopula")
d1.apply_model("CTGAN")
d1.apply_model("CopulaGAN")
d1.apply_model("TVAE")

Constrainsts created successfully!
Execution Time: 0.0004 seconds
Execution Time for training GaussianCopula: 0.1449 seconds
Execution Time for training CTGAN: 49.0681 seconds
Execution Time for training CopulaGAN: 45.322 seconds
Execution Time for training TVAE: 16.8724 seconds


### Generate Synthetic Data

In [10]:
with_constraint_GaussianCopula = d1.generate("GaussianCopula", num_rows=2000)
with_constraint_CTGAN = d1.generate("CTGAN", num_rows=2000)
with_constraint_CopulaGAN = d1.generate("CopulaGAN", num_rows=2000)
with_constraint_TVAE = d1.generate("TVAE", num_rows=2000)

Sampling rows: 100%|██████████████████████| 2000/2000 [00:00<00:00, 7870.86it/s]


Synthetic data generated successfully with GaussianCopula model!
Execution Time: 0.5882 seconds


Sampling rows: 100%|██████████████████████| 2000/2000 [00:00<00:00, 4870.28it/s]


Synthetic data generated successfully with CTGAN model!
Execution Time: 0.7355 seconds


Sampling rows: 100%|██████████████████████| 2000/2000 [00:00<00:00, 3787.43it/s]


Synthetic data generated successfully with CopulaGAN model!
Execution Time: 0.8354 seconds


Sampling rows: 100%|██████████████████████| 2000/2000 [00:00<00:00, 4941.06it/s]


Synthetic data generated successfully with TVAE model!
Execution Time: 0.7208 seconds


### Detect Relationships on Synthetic Data

### Synthetic Data Generated using GaussianCopula with constraints

In [11]:
d2 = SD_generator(with_constraint_GaussianCopula, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d2.preprocess()
d2.detect_inequality()
d2.detect_arithmetic_equality()
d2.detect_arithmetic_inequality()
d2.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0015 seconds
9 relationships detected
Execution Time: 0.6241 seconds
1 relationships detected
Execution Time: 1.5081 seconds
9 relationships detected
Execution Time: 0.7043 seconds
2 relationships detected
Execution Time: 0.5396 seconds


### New Relationships Detected:

In [12]:
print("Inequality: ", d2.inequality_dict)
print("Arithmetic Equality: ", d2.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d2.arithmetic_inequality_dict)
print("Inclusive: ", d2.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Credit_limit': ['Date_of_birth', 'Statement_balance', 'Available_credit'], 'Opening': ['Expiry_date', 'Credit_limit'], 'Expiry': ['Opening']}
Arithmetic Equality:  {'Credit_limit': [['Statement_balance', 'Available_credit']]}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2010701200000000.0]], 'Opening_date': [['Date_of_birth', 6719.007703743075]], 'Expiry_date': [['Opening_date', 1829.8046168126239]], 'Credit_limit': [['Date_of_birth', -24247.638740689603], ['Statement_balance', 38.52344068880484], ['Available_credit', 782.4400000000023]], 'Opening': [['Expiry_date', 20065408.0], ['Credit_limit', 19943185.313415647]], 'Expiry': [['Opening', 50134.0]]}
Inclusive:  {'Client_id': [['Opening', 0], ['Expiry', 8]]}


### Synthetic Data Generated using CTGAN with constraints

In [13]:
d3 = SD_generator(with_constraint_CTGAN, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d3.preprocess()
d3.detect_inequality()
d3.detect_arithmetic_equality()
d3.detect_arithmetic_inequality()
d3.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0028 seconds
8 relationships detected
Execution Time: 0.6143 seconds
1 relationships detected
Execution Time: 1.5142 seconds
8 relationships detected
Execution Time: 0.6134 seconds
2 relationships detected
Execution Time: 0.5045 seconds


### New Relationships Detected:

In [14]:
print("Inequality: ", d3.inequality_dict)
print("Arithmetic Equality: ", d3.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d3.arithmetic_inequality_dict)
print("Inclusive: ", d3.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Opening': ['Expiry_date', 'Credit_limit'], 'Expiry': ['Opening'], 'Credit_limit': ['Statement_balance', 'Available_credit']}
Arithmetic Equality:  {'Credit_limit': [['Statement_balance', 'Available_credit']]}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2000166500000000.0]], 'Opening_date': [['Date_of_birth', 6643.672358613425]], 'Expiry_date': [['Opening_date', 1850.7252087199886]], 'Opening': [['Expiry_date', 19962357.0], ['Credit_limit', 19860600.30662857]], 'Expiry': [['Opening', 50020.0]], 'Credit_limit': [['Statement_balance', 3.9400000000023283], ['Available_credit', 7.259999999998399]]}
Inclusive:  {'Client_id': [['Opening', 0], ['Expiry', 8]]}


### Synthetic Data Generated using CopulaGAN with constraints

In [15]:
d4 = SD_generator(with_constraint_CopulaGAN, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d4.preprocess()
d4.detect_inequality()
d4.detect_arithmetic_equality()
d4.detect_arithmetic_inequality()
d4.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0021 seconds
8 relationships detected
Execution Time: 0.6216 seconds
1 relationships detected
Execution Time: 1.4264 seconds
8 relationships detected
Execution Time: 0.6317 seconds
2 relationships detected
Execution Time: 0.5591 seconds


### New Relationships Detected:

In [16]:
print("Inequality: ", d4.inequality_dict)
print("Arithmetic Equality: ", d4.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d4.arithmetic_inequality_dict)
print("Inclusive: ", d4.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Opening': ['Expiry_date', 'Credit_limit'], 'Expiry': ['Opening'], 'Credit_limit': ['Statement_balance', 'Available_credit']}
Arithmetic Equality:  {'Credit_limit': [['Statement_balance', 'Available_credit']]}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2000104000000000.0]], 'Opening_date': [['Date_of_birth', 6599.268115953222]], 'Expiry_date': [['Opening_date', 1868.4476762576742]], 'Opening': [['Expiry_date', 19961858.0], ['Credit_limit', 19859252.78018252]], 'Expiry': [['Opening', 50023.0]], 'Credit_limit': [['Statement_balance', 4.040169330561184], ['Available_credit', 7.6599999999998545]]}
Inclusive:  {'Client_id': [['Opening', 0], ['Expiry', 8]]}


### Synthetic Data Generated using TVAE with constraints

In [17]:
d5 = SD_generator(with_constraint_TVAE, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d5.preprocess()
d5.detect_inequality()
d5.detect_arithmetic_equality()
d5.detect_arithmetic_inequality()
d5.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0028 seconds
8 relationships detected
Execution Time: 0.5847 seconds
1 relationships detected
Execution Time: 1.4133 seconds
8 relationships detected
Execution Time: 0.6095 seconds
2 relationships detected
Execution Time: 0.3872 seconds


### New Relationships Detected:

In [18]:
print("Inequality: ", d5.inequality_dict)
print("Arithmetic Equality: ", d5.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d5.arithmetic_inequality_dict)
print("Inclusive: ", d5.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Opening': ['Expiry_date', 'Credit_limit'], 'Expiry': ['Opening'], 'Credit_limit': ['Statement_balance', 'Available_credit']}
Arithmetic Equality:  {'Credit_limit': [['Statement_balance', 'Available_credit']]}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2000052900000000.0]], 'Opening_date': [['Date_of_birth', 6604.18468302094]], 'Expiry_date': [['Opening_date', 1826.7241858785856]], 'Opening': [['Expiry_date', 19961832.0], ['Credit_limit', 19880690.651289474]], 'Expiry': [['Opening', 50017.0]], 'Credit_limit': [['Statement_balance', 3.9400000000023283], ['Available_credit', 7.259999999994761]]}
Inclusive:  {'Client_id': [['Opening', 0], ['Expiry', 8]]}


## Generate Synthetic Data without Constraints

In [19]:
d6 = SD_generator(data, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d6.preprocess()
d6.detect_inequality()
d6.detect_arithmetic_equality()
d6.detect_arithmetic_inequality()
d6.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0024 seconds
8 relationships detected
Execution Time: 0.5929 seconds
1 relationships detected
Execution Time: 1.4233 seconds
8 relationships detected
Execution Time: 0.6011 seconds
2 relationships detected
Execution Time: 0.4761 seconds


### Original Relationships Detected:

In [20]:
print("Inequality: ", d6.inequality_dict)
print("Arithmetic Equality: ", d6.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d6.arithmetic_inequality_dict)
print("Inclusive: ", d6.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Opening': ['Expiry_date', 'Credit_limit'], 'Expiry': ['Opening'], 'Credit_limit': ['Statement_balance', 'Available_credit']}
Arithmetic Equality:  {'Credit_limit': [['Statement_balance', 'Available_credit']]}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2000010600000000.0]], 'Opening_date': [['Date_of_birth', 6599.0]], 'Expiry_date': [['Opening_date', 1826.0]], 'Opening': [['Expiry_date', 19961750.0], ['Credit_limit', 19900301.0]], 'Expiry': [['Opening', 49999.0]], 'Credit_limit': [['Statement_balance', 3.9400000000023283], ['Available_credit', 7.260000000000218]]}
Inclusive:  {'Client_id': [['Opening', 0], ['Expiry', 8]]}


### Create Models

In [21]:
d6.create_constraints(inequality=False, arithmetic_equality=False, arithmetic_inequality=False, inclusive=False)
d6.apply_model("GaussianCopula")
d6.apply_model("CTGAN")
d6.apply_model("CopulaGAN")
d6.apply_model("TVAE")

Constrainsts created successfully!
Execution Time: 0.0 seconds
Execution Time for training GaussianCopula: 0.1546 seconds
Execution Time for training CTGAN: 45.6992 seconds
Execution Time for training CopulaGAN: 47.4916 seconds
Execution Time for training TVAE: 16.9435 seconds


### Generate Synthetic Data

In [22]:
without_constraint_GaussianCopula = d6.generate("GaussianCopula", num_rows=2000)
without_constraint_CTGAN = d6.generate("CTGAN", num_rows=2000)
without_constraint_CopulaGAN = d6.generate("CopulaGAN", num_rows=2000)
without_constraint_TVAE = d6.generate("TVAE", num_rows=2000)

Synthetic data generated successfully with GaussianCopula model!
Execution Time: 0.0701 seconds
Synthetic data generated successfully with CTGAN model!
Execution Time: 0.1099 seconds
Synthetic data generated successfully with CopulaGAN model!
Execution Time: 0.218 seconds
Synthetic data generated successfully with TVAE model!
Execution Time: 0.0913 seconds


### Detect Relationships on Synthetic Data

### Synthetic Data Generated using GaussianCopula without constraints

In [23]:
d7 = SD_generator(without_constraint_GaussianCopula, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d7.preprocess()
d7.detect_inequality()
d7.detect_arithmetic_equality()
d7.detect_arithmetic_inequality()
d7.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.003 seconds
8 relationships detected
Execution Time: 0.6561 seconds
0 relationships detected
Execution Time: 1.4388 seconds
8 relationships detected
Execution Time: 0.6199 seconds
0 relationships detected
Execution Time: 0.4844 seconds


### New Relationships detected:

In [24]:
print("Inequality: ", d7.inequality_dict)
print("Arithmetic Equality: ", d7.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d7.arithmetic_inequality_dict)
print("Inclusive: ", d7.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Opening': ['Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit'], 'Expiry': ['Opening']}
Arithmetic Equality:  {}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2000053299291494.0]], 'Opening_date': [['Date_of_birth', 1080.0]], 'Expiry_date': [['Opening_date', 1820.0]], 'Opening': [['Expiry_date', 19961923.0], ['Credit_limit', 19904811.0], ['Statement_balance', 19914246.26], ['Available_credit', 19903857.66328935]], 'Expiry': [['Opening', 48909.0]]}
Inclusive:  {}


### Synthetic Data Generated using CTGAN without constraints

In [25]:
d8 = SD_generator(without_constraint_CTGAN, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d8.preprocess()
d8.detect_inequality()
d8.detect_arithmetic_equality()
d8.detect_arithmetic_inequality()
d8.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0025 seconds
14 relationships detected
Execution Time: 0.6088 seconds
0 relationships detected
Execution Time: 1.2559 seconds
14 relationships detected
Execution Time: 1.0543 seconds
0 relationships detected
Execution Time: 0.5775 seconds


### New Relationships detected:

In [26]:
print("Inequality: ", d8.inequality_dict)
print("Arithmetic Equality: ", d8.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d8.arithmetic_inequality_dict)
print("Inclusive: ", d8.inclusive_dict)

Inequality:  {'Client_id': ['Opening', 'Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Date_of_birth'], 'Opening': ['Opening_date', 'Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit'], 'Expiry': ['Opening_date', 'Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit']}
Arithmetic Equality:  {}
Arithmetic Inequality:  {'Client_id': [['Opening', 2000010599828876.0], ['Expiry', 2000010599778876.0]], 'Opening_date': [['Date_of_birth', -1242.0]], 'Expiry_date': [['Date_of_birth', 585.0]], 'Opening': [['Opening_date', 19955183.0], ['Expiry_date', 19953710.0], ['Credit_limit', 19900106.0], ['Statement_balance', 19901941.96], ['Available_credit', 19901337.67]], 'Expiry': [['Opening_date', 20005183.0], ['Expiry_date', 20003387.0], ['Credit_limit', 19950106.0], ['Statement_balance', 19951941.96], ['Available_credit', 19951337.67]]}
Inclusive:  {}


### Synthetic Data Generated using CopulaGAN without constraints

In [27]:
d9 = SD_generator(without_constraint_CopulaGAN, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d9.preprocess()
d9.detect_inequality()
d9.detect_arithmetic_equality()
d9.detect_arithmetic_inequality()
d9.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0018 seconds
14 relationships detected
Execution Time: 0.6008 seconds
0 relationships detected
Execution Time: 1.2722 seconds
14 relationships detected
Execution Time: 1.0467 seconds
0 relationships detected
Execution Time: 0.5429 seconds


### New Relationships detected:

In [28]:
print("Inequality: ", d9.inequality_dict)
print("Arithmetic Equality: ", d9.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d9.arithmetic_inequality_dict)
print("Inclusive: ", d9.inclusive_dict)

Inequality:  {'Client_id': ['Opening', 'Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Date_of_birth'], 'Opening': ['Opening_date', 'Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit'], 'Expiry': ['Opening_date', 'Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit']}
Arithmetic Equality:  {}
Arithmetic Inequality:  {'Client_id': [['Opening', 2000010600509897.0], ['Expiry', 2000010600608296.0]], 'Opening_date': [['Date_of_birth', 118.0]], 'Expiry_date': [['Date_of_birth', 1062.0]], 'Opening': [['Opening_date', 19956139.0], ['Expiry_date', 19953579.0], ['Credit_limit', 19906405.0], ['Statement_balance', 19918991.9], ['Available_credit', 19915864.379119765]], 'Expiry': [['Opening_date', 20005375.0], ['Expiry_date', 20003698.0], ['Credit_limit', 19951779.0], ['Statement_balance', 19955402.65], ['Available_credit', 19960858.64206941]]}
Inclusive:  {}


### Synthetic Data Generated using TVAE without constraints

In [29]:
d10 = SD_generator(without_constraint_TVAE, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d10.preprocess()
d10.detect_inequality()
d10.detect_arithmetic_equality()
d10.detect_arithmetic_inequality()
d10.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0028 seconds
8 relationships detected
Execution Time: 0.6112 seconds
0 relationships detected
Execution Time: 1.3565 seconds
8 relationships detected
Execution Time: 0.6089 seconds
0 relationships detected
Execution Time: 0.3786 seconds


### New Relationships detected:

In [30]:
print("Inequality: ", d10.inequality_dict)
print("Arithmetic Equality: ", d10.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d10.arithmetic_inequality_dict)
print("Inclusive: ", d10.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Opening': ['Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit'], 'Expiry': ['Opening']}
Arithmetic Equality:  {}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2000010599998501.0]], 'Opening_date': [['Date_of_birth', 3452.0]], 'Expiry_date': [['Opening_date', -1147.0]], 'Opening': [['Expiry_date', 19960915.0], ['Credit_limit', 19900106.0], ['Statement_balance', 19917119.23], ['Available_credit', 19917620.373674862]], 'Expiry': [['Opening', -81205.0]]}
Inclusive:  {}
